# Task 4
Creating a video by predicting ID of cards in the given video file

##### Importing necessary modules

In [1]:
import tensorflow as tf
import numpy as np
import cv2

##### Importing the task 3 model for prediction

In [2]:
task3_model = tf.keras.models.load_model('output_files/iva_task3_transfer.h5')
def input_image_pipeline(img, height=224, width=224):
    '''
    Pipeline function to convert the localized card for prediction
    '''
    try:
        res = cv2.resize(img, (width, height), interpolation = cv2.INTER_AREA) # resize it in (224 x 244)
        return res, True # True when resize was successfull
    except:
        return [], False

##### Function to get the card if it is tilted

In [3]:
def tilted_img_crop(frame, rect):
    '''
    Returns the cropped image and the bounding box points
    frame -> image in which there is a bounding box
    rect -> rectangle points with angle
    '''
    angle = rect[2] # Get the angle of the rectangle
    rows,cols = frame.shape[0], frame.shape[1] # Get the rows and col of frame
    M = cv2.getRotationMatrix2D((cols/2,rows/2),angle,1) # Create a rotational matrix using cv2 API
    frame_rot = cv2.warpAffine(frame,M,(cols,rows)) # Rotate the frame using M and shape of the frame
    box = cv2.boxPoints(rect) # Get the bounding boxes
    pts = np.int0(cv2.transform(np.array([box]), M))[0] # Get 4 rectangluar points
    pts[pts < 0] = 0 
    crop = frame_rot[pts[1][1]:pts[0][1], pts[1][0]:pts[2][0]]
    return crop, pts

##### Do the prediction on video-001.mov

In [4]:
data_dir = 'iva_files/DATA/'
results_dir = 'iva_files/results/'
first_video = data_dir + 'video-001.MOV'
cap = cv2.VideoCapture(first_video) # Read the video frames
total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) # Just for stat purpose
print('TOTAL FRAMES -',total)
res_video = [] # Saving each frame's prediction
print("----------- Processing the frames -----------")
count = 0
while(cap.isOpened()):
    ret, frame = cap.read()
    if(ret):
        count += 1
        # Applying task used in get_test_cards.ipynb
        low_green = np.array([25, 52, 72])
        high_green = np.array([102, 255, 255])
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv, low_green, high_green)
        mask = 255-mask
        res = cv2.bitwise_and(frame, frame, mask=mask)
        contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        for i in range(len(contours)):
            area = cv2.contourArea(contours[i])
            if(area > 30000):
                rect = cv2.minAreaRect(contours[i])
                card, pts = tilted_img_crop(frame, rect)
                box = cv2.boxPoints(rect)
                cbox = np.int0(box)
                card, isvalid = input_image_pipeline(card) # Preproccessing the card
                if(isvalid == False):
                    continue
                # Predicting the ID of the preprocessed card
                id = np.argmax(task3_model.predict(tf.expand_dims(card, axis=0))) 
                pos = (cbox[2][0], cbox[2][1])
                # Writing the prediction above the localized card.
                cv2.putText(frame, str(id), pos, cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 127, 255), 4, cv2.LINE_AA) 
                cv2.drawContours(frame,[cbox],0,(255,0,255),4)
        res_video.append(frame)
        if(count%10==0):
            print(count, 'frames processed')
    else:
        break
print('---------------------------------------------')
cap.release()
res_video = np.array(res_video)

TOTAL FRAMES - 217
----------- Processing the frames -----------
10 frames processed
20 frames processed
30 frames processed
40 frames processed
50 frames processed
60 frames processed
70 frames processed
80 frames processed
90 frames processed
100 frames processed
110 frames processed
120 frames processed
130 frames processed
140 frames processed
150 frames processed
160 frames processed
170 frames processed
180 frames processed
190 frames processed
200 frames processed
210 frames processed
---------------------------------------------


##### Saving the result for video-001 in task4_result4.avi video file

In [5]:
first_video_result = results_dir + 'task4_result1.avi'
size = (res_video[1].shape[1], res_video[1].shape[0])
print('Writing the frames in task4_result1.avi video file -', end=" ")
# Building the object to write it as a video file
out = cv2.VideoWriter(first_video_result,cv2.VideoWriter_fourcc(*'DIVX'), 15, size) 
for i in range(len(res_video)):
    out.write(res_video[i])
out.release()
print('Done')

Writing the frames in task4_result1.avi video file - Done


In [6]:
# Same explanation on video-001 video prediction
second_video = data_dir + 'video-002.MOV'
cap = cv2.VideoCapture(second_video)
total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print('TOTAL FRAMES -',total)
res_video = []
print("----------- Processing the frames -----------")
count = 0
while(cap.isOpened()):
    ret, frame = cap.read()
    if(ret):
        count += 1
        low_green = np.array([25, 52, 72])
        high_green = np.array([102, 255, 255])
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv, low_green, high_green)
        mask = 255-mask
        res = cv2.bitwise_and(frame, frame, mask=mask)
        contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        for i in range(len(contours)):
            area = cv2.contourArea(contours[i])
            if(area > 30000):
                rect = cv2.minAreaRect(contours[i])
                card, pts = tilted_img_crop(frame, rect)
                box = cv2.boxPoints(rect)
                cbox = np.int0(box)
                card, isvalid = input_image_pipeline(card)
                if(isvalid == False):
                    continue
                id = np.argmax(task3_model.predict(tf.expand_dims(card, axis=0)))
                pos = (cbox[2][0], cbox[2][1])
                cv2.putText(frame, str(id), pos, cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 127, 255), 4, cv2.LINE_AA)
                cv2.drawContours(frame,[cbox],0,(255,0,255),4)
        res_video.append(frame)
        if(count%10==0):
            print(count, 'frames processed')
    else:
        break
print('---------------------------------------------')
cap.release()
res_video = np.array(res_video)

TOTAL FRAMES - 524
----------- Processing the frames -----------
10 frames processed
20 frames processed
30 frames processed
40 frames processed
50 frames processed
60 frames processed
70 frames processed
80 frames processed
90 frames processed
100 frames processed
110 frames processed
120 frames processed
130 frames processed
140 frames processed
150 frames processed
160 frames processed
170 frames processed
180 frames processed
190 frames processed
200 frames processed
210 frames processed
220 frames processed
230 frames processed
240 frames processed
250 frames processed
260 frames processed
270 frames processed
280 frames processed
290 frames processed
300 frames processed
310 frames processed
320 frames processed
330 frames processed
340 frames processed
350 frames processed
360 frames processed
370 frames processed
380 frames processed
390 frames processed
400 frames processed
410 frames processed
420 frames processed
430 frames processed
440 frames processed
450 frames processed

In [7]:
second_video_result = results_dir + 'task4_result2.avi'
size = (res_video[1].shape[1], res_video[1].shape[0])
print('Writing the frames in task4_result2.avi video file -', end=" ")
out = cv2.VideoWriter(second_video_result,cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
for i in range(len(res_video)):
    out.write(res_video[i])
out.release()
print('Done')

Writing the frames in task4_result2.avi video file - Done
